# Script to generate predictions of KO strain design performance

## Detailed in "Integrated knowledge mining, genome-scale modeling, and machine learning for predicting *Yarrowia lipolytica* bioproduction".

### Description. 
Script takes a target compound, a list of reactions or genes to test for KO or OE, and generates predicted titers for each design. 

The default conditions for prediction are in glucose, with no prior genetic engineering. The prediction is obtained through generated machine-learning model detailed in the publication.

#### Procedure:
1. Read in data and constructs to screen from "Template_ComputationalDesign" spreadsheet.
2. Generate FBA features for the WT and each strain construct.
3. Predict the titer of each strain.
4. Output the results.

#### Inputs:
1. Supplemental Excel File 6- CSD Template.xlsx: Spreadsheet where the product, testing environmental conditons, and list of KO targets to screen are input.
      Supplemental Excel File 6- CSD Template.xslx
2. Data Encoding File 
      Supplemental Data File 2- DataCharateristics & Encoding.xlsx

#### Output:
1. titerPredictionsKO.csv: Spreadsheet containing a prediction of the WT strain titer, each KO strain titer, and the FBA predicted product yield and biomass growth rate.


#### Additional required scripts:
1. FBA_function_.py:
    Performs FBA feature generation and extraction
2. encodingFunction_.py:
    Encodes the data for input to the ML model
3. FBA_functionOE_.py:
    Performs Gene OE feature generation and extraction

### Libraries to import

In [1]:
# perform OE or KO?
perform_knockouts = 0 # 0 for overexpression

In [5]:
import pandas as pd
import pickle
from collections import defaultdict
import warnings
import numpy as np
import os

from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer,MinMaxScaler, MaxAbsScaler,Normalizer,PowerTransformer
from sklearn.pipeline import Pipeline
import warnings
from sklearn.linear_model import ElasticNet,Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from mlxtend.regressor import StackingRegressor
from sklearn.model_selection import learning_curve

if perform_knockouts==1:
    #custom functions provided in the directory.
    from FBA_function_ import FBA_FeatureExtraction #KO
else:
    #custom functions provided in the directory for OE.
    from FBA_functionOE_ import FBA_FeatureExtraction #OE


# from FBA_function_ import FBA_FeatureExtraction
from encodingFunction_ import encodeTransform

### Ensure the spreadsheet is within the directory.

In [6]:
dir_path = os.path.dirname(os.path.realpath('Supplemental Excel File 6- CSD Template.xlsx'))
file_path = os.path.join(dir_path,'Supplemental Excel File 6- CSD Template.xlsx')

In [7]:
#reads in the information from the datasheets
raw_construct = pd.read_excel(file_path,sheet_name='predictions',skiprows=range(2))
optKnockRxns = pd.read_excel(file_path,sheet_name='targetRxns')

optOERxns = optKnockRxns ## delete


# consolidate meta-information into usable features
data = raw_construct
optData = optKnockRxns
data['number_genes_mod'] = data.genes_modified_updated.apply(lambda x: x.count(';')+1 if isinstance(x,str) else 0)
data['number_genes_deleted'] = data.gene_deletion.apply(lambda x: x.replace(';','').count('1') if isinstance(x,str) else 0)
data['number_total_genes_overexp'] = data.gene_overexpression.apply(lambda x: x.replace(';','').count('1') if isinstance(x,str) else 0)
data['number_genes_het'] = data.heterologous_gene.apply(lambda x: x.replace(';','').count('1') if isinstance(x,str) else 0)

# hettemp1 = data.heterologous_gene#.apply(lambda x: x if isinstance(x,str) else 'NA')
hettemp1 = data.heterologous_gene.apply(lambda x: x if isinstance(x,str) else 'NA')
data.heterologous_gene
hettemp2 = hettemp1.str.split(';',expand=True)

# overexpressTemp1 = data.gene_overexpression.fillna('2')
overexpressTemp1 = data.gene_overexpression.apply(lambda x: x if isinstance(x,str) else 'NA')
overexpressTemp2 = overexpressTemp1.str.split(';',expand=True)
nativeGenes = overexpressTemp2[hettemp2=='0']

data['number_native_genes_overexp'] = nativeGenes.count(axis=1)


## FBA Modeling

In [8]:
##FBA modeling
#GSM to use, default is 'iYLI647'
FBA_models=['iYLI647']

#Extracted FBA features
if perform_knockouts==1:
    output = FBA_FeatureExtraction(data,optKnockRxns,FBA_models) #KO
else:
    output = FBA_FeatureExtraction(data,optKnockRxns,optKnockRxns,FBA_models) # OE



/home/jjczajka/Documents/GitHub/YlipCSD/PredictionFiles/FBA_functionOE_.py:46 UserWarning: DataFrame columns are not unique, some columns will be omitted.
/home/jjczajka/Documents/GitHub/YlipCSD/PredictionFiles/FBA_functionOE_.py:61 UserWarning: DataFrame columns are not unique, some columns will be omitted.


1.1398166174414033 iYLI647
Completed  1  overexpression simulations
0 OE failures
0 Prod failures
0 0 0 0 0 0 failure cases 1-6


In [9]:
output

,paper_number,blank1,cs1,cs_conc1,cs1_heatCombustion(kJ/mol),cs2,cs_conc2,cs2_heatCombustion(kJ/mol),reactor_type,rxt_volume,...,EMP_iYLI647,PPP_iYLI647,TCA_iYLI647,NADPH_iYLI647,ATP_iYLI647,PrdtFlux_iYLI647,PrdtYield_iYLI647,Biomass_iYLI647,O2Uptake_iYLI647,GlcUptake_iYLI647
0,1,NaN,10,20,34734.5,0,0,0,1,0.05,...,6.667322,7.343305,1.215066,15.982695,56.429296,0.210230,0.125476,0.854862,-13.207577,-10.0
1,1,NaN,10,20,34734.5,0,0,0,1,0.05,...,6.667322,7.343305,1.215066,15.982695,56.510524,0.210230,0.125476,0.854862,-13.207577,-10.0
2,1,NaN,10,20,34734.5,0,0,0,1,0.05,...,6.667322,7.343305,1.215066,15.982695,56.587441,0.210230,0.125476,0.854862,-13.207577,-10.0
3,1,NaN,10,20,34734.5,0,0,0,1,0.05,...,6.677714,7.280954,3.321615,15.985664,56.962038,0.205776,0.122817,0.854862,-13.483700,-10.0


## Encode data

In [10]:
#encode data, using output from FBA modeling section
encodedData = encodeTransform(output)

### Features used in the machine learning trained model.

In [11]:
#rename feature to original feature name
encodedData['mw_Lipids'] = encodedData['mw']

cols_train__set = [
'mw_Lipids'
,'pH'
,'product_deltaGo'
,'foldCarbonFed'
,'product_name'
,'rxt_volume'
,'inputThermo(kJ/L)'
,'FermentationTime'
,'atp_cost'
,'precursorsRequiredEncoded'
,'nadh_nadph_cost'
,'Pathway_enzymatic_steps'
,'averageThermBarrier'
,'media'
,'number_genes_het'
,'number_native_genes_overexp'
,'ATP_iYLI647'
,'NADPH_iYLI647'
,'PPP_iYLI647'
,'TCA_iYLI647'
,'PrdtYield_iYLI647'
]

warnings.simplefilter('ignore')


useful_cols = []
useful_cols.extend(cols_train__set)
data = pd.DataFrame()


data = encodedData.loc[:,useful_cols]
for column in data:
    data[column] = data[column].astype(np.float32)

In [12]:
#obtain features used from the data
warnings.simplefilter('ignore')


useful_cols = []
useful_cols.extend(cols_train__set)
data = pd.DataFrame()

data = encodedData.loc[:,useful_cols]
for column in data:
    data[column] = data[column].astype(np.float32)


In [13]:
#open the ML model for prediction
with open('M21iYL.pickle','rb') as f:
    masterGrid = pickle.load(f)

masterGrid = masterGrid[0]

In [14]:
#perform prediction on data
x_testData = data.copy()
target = 'Product_titer(g/L)'
x_testData.PrdtYield_iYLI647.fillna(0,inplace=True)

#prediction
y_prediction = np.exp(masterGrid[target].predict(x_testData))
len(y_prediction)

4

In [15]:
x_testData

,mw_Lipids,pH,product_deltaGo,foldCarbonFed,product_name,rxt_volume,inputThermo(kJ/L),FermentationTime,atp_cost,precursorsRequiredEncoded,...,Pathway_enzymatic_steps,averageThermBarrier,media,number_genes_het,number_native_genes_overexp,ATP_iYLI647,NADPH_iYLI647,PPP_iYLI647,TCA_iYLI647,PrdtYield_iYLI647
0,596.849976,0.0,2575.899902,0.0,4.0,2.0,502.597321,120.0,24.0,24.0,...,16.0,271.0,3.0,3.0,0.0,56.429295,15.982695,7.343305,1.215066,0.125476
1,596.849976,0.0,2575.899902,0.0,4.0,2.0,502.597321,120.0,24.0,24.0,...,16.0,271.0,3.0,3.0,0.0,56.510525,15.982695,7.343305,1.215066,0.125476
2,596.849976,0.0,2575.899902,0.0,4.0,2.0,502.597321,120.0,24.0,24.0,...,16.0,271.0,3.0,3.0,0.0,56.587440,15.982695,7.343305,1.215066,0.125476
3,596.849976,0.0,2575.899902,0.0,4.0,2.0,502.597321,120.0,24.0,24.0,...,16.0,271.0,3.0,3.0,0.0,56.962036,15.985663,7.280954,3.321615,0.122817


In [16]:
#output dataframe
MLOutput = pd.DataFrame()
MLOutput['TiterPrediction(g/L)'] = y_prediction
MLOutput['FBA predicted Biomass'] = output['Biomass_iYLI647']
MLOutput['FBA predicted Yield'] = data['PrdtYield_iYLI647']
# MLOutput.at['rxns'] = optKnockRxns
MLOutput.index = data.index


In [26]:
#Save file
MLOutput.to_csv('Screen.csv')

In [17]:
MLOutput

,TiterPrediction(g/L),FBA predicted Biomass,FBA predicted Yield
0,0.008105,0.854862,0.125476
1,0.008105,0.854862,0.125476
2,0.008105,0.854862,0.125476
3,0.008986,0.854862,0.122817
